In [66]:
#importing necessary modules
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import codecs
import math
from IPython.display import clear_output
import operator
from collections import defaultdict
sns.set()

In [28]:
#reading in excel file to pandas dataframe
df_1=pd.read_excel('./data/jester-data-1.xls',header=None)
df_2=pd.read_excel('./data/jester-data-2.xls',header=None)
df_3=pd.read_excel('./data/jester-data-3.xls',header=None)
frames = [df_1, df_2, df_3]
df = pd.concat(frames)

In [29]:
#reading in excel file to pandas dataframe
df=df.rename(columns = {0:'jokes_rated'})
df=df.reset_index()
df=df.drop(['index','jokes_rated'],axis=1)
df=df.reset_index()

In [30]:
#setting name of first column to the number of jokes rated
#now each row is a single user and each column is a joke and the values are a users ratings of that joke
df.rename(columns={'index':'user_id'}, inplace=True)
df=df.set_index('user_id')


In [31]:
df=df.replace(to_replace=99.00, value=np.nan)

In [32]:
df.head()

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
user_id,,,,,,,,,,,,,,,,,,,,,
0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,...,2.82,NaN,NaN,NaN,NaN,NaN,-5.63,NaN,NaN,NaN
1,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,NaN,NaN,NaN,NaN,9.03,9.27,9.03,9.27,NaN,NaN,...,NaN,NaN,NaN,9.08,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,8.35,NaN,NaN,1.80,8.16,-2.82,6.21,NaN,1.84,...,NaN,NaN,NaN,0.53,NaN,NaN,NaN,NaN,NaN,NaN
4,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


## Joke recomender

In this notebook I will be building a joke recommender using the jester data set. To do so, I will be using the principles of both user-based and item-based collaborative filtering.  I will start first with user based


#### User Based Collaborative filtering

In [56]:
#SCRATCH PAD FOR WORK
user_id=0
user_rated=[joke for joke,rating in df.loc[user_id].iteritems() if not math.isnan(rating)] 
print(user_rated)
user=df.loc[user_id]

similarites=df.dot(user)
sorted_similar=similarites.sort_values(ascending=False)


found_joke=False
for other_user_id,similarity in sorted_similar.iteritems():
    
    if(found_joke==False): 
        other_user_rated_positive=[joke for joke,rating in df.loc[other_user_id].iteritems() if rating >0]
        list_of_new_jokes=list(set(other_user_rated_positive)-set(user_rated))
        #print(list_of_new_jokes)
        if(len(list_of_new_jokes)!=0):
            
            new_joke_num=list_of_new_jokes[0]
            found_joke=True
print(found_joke)        
filename="./data/jokes/init"+str(new_joke_num)+".html"
f=codecs.open(filename, 'r')
html = f.read()
start=html.find('<!--begin of joke -->')
end=html.find('<!--end of joke -->')
string=html[start+len('<!--begin of joke -->'):end]
print(string)  

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 78, 82, 91, 97]
True

Age and Womanhood
<p>
1. Between the ages of 13 and 18 ...<br>
&nbsp;&nbsp;&nbsp;&nbsp;<i>She is like Africa, virgin and unexplored.</i> 
<p>
2. Between the ages of 19 and 35 ...<br>
&nbsp;&nbsp;&nbsp;&nbsp;<i>She is like Asia, hot and exotic.</i> 
<p>
3. Between the ages of 36 and 45 ...<br>
&nbsp;&nbsp;&nbsp;&nbsp;<i>She is like America, fully explored, breathtakingly beautiful,and free with her resources.</i>
<p>
4. Between the ages of 46 and 56 ...<br>
&nbsp;&nbsp;&nbsp;&nbsp;<i>She is like Europe, exhausted but still has points of interest.</i> 
<p>
5. After 56 she is like Australia ...<br>
&nbsp;&nbsp;&nbsp;&nbsp;<i>Everybody knows it's down there, but who gives a damn?</i>



In [62]:
def new_joke_user_filtering(df,user_id):
    #user based collaborative filtering
    
    #obtaining list of jokes rated by the user we are investigating
    user_rated=[joke for joke,rating in df.loc[user_id].iteritems() if not math.isnan(rating)] 
    #grabbing row of the user_id from data frame
    user=df.loc[user_id]
    
    #calculating cosine similarity between user and all other users
    similarities=df.dot(user)
    
    #sorting by most similar users first
    sorted_similar=similarities.sort_values(ascending=False)
    
    #condition to determine when to stop
    found_joke=False
    
    #iterating through all user sorted by most similar first
    for other_user_id,similarity in sorted_similar.iteritems():
        
        if(found_joke==False): 
            
            #obtaining list of jokes rated by other user
            other_user_rated_positive=[joke for joke,rating in df.loc[other_user_id].iteritems() if not math.isnan(rating) and rating >0.00]
            #finding jokes that our user hasnt yet seen
            list_of_new_jokes=list(set(other_user_rated_positive)-set(user_rated))
            
            #determining if any unseen jokes exist between our user and this other similar user
            if(len(list_of_new_jokes)!=0):
                new_joke_num=list_of_new_jokes[0]
                found_joke=True
        else:
            break
    
    
    #creating return object to hold the new joke and the joke number
    obj_return=(return_joke(new_joke_num),new_joke_num)
    return obj_return
    
    

In [63]:
def funniest():
    #function to return the top 5 jokes and their associate joke number
    funniest=df.mean(axis=0)
    jokes_and_rating=funniest.sort_values(ascending=False)

    #selecting top and bottom 5 jokes
    top_5=jokes_and_rating[:5]

    #creating object to return
    return_list=[]
    
    #iterating through top 5 jokes
    x=1
    for num,rating in top_5.iteritems():
    
        #formatting and scrapping of html file
        filename="./data/jokes/init"+str(num)+".html"
        f=codecs.open(filename, 'r')
        html = f.read()
        start=html.find('<!--begin of joke -->')
        end=html.find('<!--end of joke -->')
        string=html[start+len('<!--begin of joke -->'):end]
        
        #formating tuple to return joke and joke number
        obj=(string,num)
        
        return_list.append(obj)
        
    return return_list
        

In [64]:
def return_joke(joke_num):
    #formating of joke 
    filename="./data/jokes/init"+str(joke_num)+".html"
    f=codecs.open(filename, 'r')
    html = f.read()
    start=html.find('<!--begin of joke -->')
    end=html.find('<!--end of joke -->')
    string=html[start+len('<!--begin of joke -->'):end]
    return string   

In [69]:
def user_based_filtering(df):
    #main function
    
    #list of averages to use in performance evaluation
    avg_list=[]
    
    #taking name to add to dataframe
    new_user = input('Enter your name: ')
    clear_output()
    
    #adding new user to data frame if it isnt already
    if(new_user not in list(df.index.values)):
        #formatting i needed to do
        add_list=[np.nan for x in range(1,102)]
        temp_df=pd.DataFrame(add_list)
        temp_df=temp_df.transpose()
        temp_df[0]=new_user
        temp_df.rename(columns={0:'user_id'}, inplace=True)
        temp_df=temp_df.set_index('user_id')
        
        #adding new user to dataframe
        df=df.append(temp_df,ignore_index=False)
    
    #getting list of jokes rated by the user to determine if they have rated any jokes yet, new users will be 
    #given the top jokes to rate to get a feel for their preferences
    jokes_rated=[joke for joke,rating in df.loc[new_user].iteritems() if not math.isnan(rating)]
    cont='yes'
    
    if(int(len(jokes_rated))==0):
        for text,joke_num in funniest():
            cont=input('Still want to hear a joke (yes to continue, no to stop):')
            clear_output()
            if(cont=='yes'):
                
                #printing joke
                print(text)
                
                #taking input
                new_rating=input('what rating would you give this joke (-10.0 to 10.0)')
                
                #placing rating data into dataframe
                if(-10.0<=int(new_rating)<=10.0):
                    df.loc[new_user,joke_num]=int(new_rating)
                else:
                    print('invalid input')
                
                clear_output()
            else:
                break
                
    #continuation process for jokes
    while(cont=='yes'):
        #print(df.loc[new_user])
        avg=df.loc[new_user].mean(axis=0)
        avg_list.append(avg)
        print('avg',avg)
        cont=input('Still want to hear a joke (yes to continue, no to stop):')
        clear_output()
        if(cont=='yes'):
            
            #getting new joke through user based collaborative filtering
            text,joke_num=new_joke_user_filtering(df,new_user)
            
            #printing joke
            print(text)
            
            #taking input
            new_rating=input('what rating would you give this joke (-10.0 to 10.0)')
            
            #placing rating data into dataframe
            if(-10.0<=int(new_rating)<=10.0):
                df.loc[new_user,joke_num]=int(new_rating)
            else:
                print('invalid input')
            clear_output()
            
    #performance evalutation through slope   
    if(len(avg_list)>=2):
        z=np.polyfit(np.arange(0,len(avg_list)),avg_list,1)
        print("slope of average over time in this instance", z[0])
    
    #need to return the data frame to keep it updated (will reset every time this notebook is restarted) 
    return df
    
#execution of main function
df=user_based_filtering(df)

slop of average over time in this instance -1.03333333333


Above we have a recommendation system for jokes based on user based collaborative filtering.  Similar jokes are calcuated after every rating of the joke to ensure the most accurate suggestions.  Now I am going to repeat this process and process an identical program with item based collaborative filtering

#### Item based collaborative filtering

In [70]:
#SCRATCH PAD FOR WORK
item_df=df.transpose()

user_id=0

user_rated = {joke:rating for (joke, rating) in df.loc[user_id].iteritems() if not math.isnan(rating)}
user_rated_positive = {joke:rating for (joke, rating) in df.loc[user_id].iteritems() if not math.isnan(rating) and rating>0}
sorted_user_rated_positive = sorted(user_rated.items(), key=operator.itemgetter(1),reverse=True)

suggestions=defaultdict(float)
found_joke=False
for joke, rating in sorted_user_rated_positive:
    joke_row=item_df.loc[joke]
    similarities=item_df.dot(joke_row)
    
    for other_joke_id, similarity in enumerate(similarities):
        suggestions[other_joke_id]+=similarity*rating


sorted_suggestions=sorted(suggestions.items(),key=operator.itemgetter(1),reverse=True)

#print(sorted_suggestions[0][0])
    



In [80]:
def new_joke_item_filtering(df,user_id):
    #returning most popular joke through item based collaboritive filtering
       
    #transposing dataframe so that jokes are rows and users are columns
    item_df=df.transpose()
    
    #creating dictionaries of joke and rating values
    user_rated = {joke:rating for (joke, rating) in df.loc[user_id].iteritems() if not math.isnan(rating)}
    user_rated_positive = {joke:rating for (joke, rating) in df.loc[user_id].iteritems() if not math.isnan(rating) and rating>0}
    #print(user_rated_positive)
    #print(type(user_rated_positive))
    
    #creating default dict of suggestions
    suggestions=defaultdict(float)
    
    #filling suggestions for relative joke importance
    for joke, rating in user_rated_positive.items():
        #calculating cosine similarity for each item
        joke_row=item_df.loc[joke]
        similarities=item_df.dot(joke_row)
        
        #here we are adding the relative similarity for each joke multiplied by the rating of the joke they are 
        #comparing it to, meaning the higher rated the joke the bigger the bonus for that similarity value
        for other_joke_id, similarity in enumerate(similarities):    
            suggestions[other_joke_id+1]+=similarity*rating

    #sorting suggestions 
    sorted_suggestions=sorted(suggestions.items(),key=operator.itemgetter(1),reverse=True)
    
    #choosing the first joke that the user hasnt already seen
    for joke_id,rating in sorted_suggestions:
        if(joke_id not in user_rated):
            new_joke_num=joke_id
            break

    #creating object with the joke and the jokes number
    obj_return=(return_joke(new_joke_num),new_joke_num)
    return obj_return


    

In [82]:
def item_based_filtering(df):
    #main function
    
    #list of averages to use in performance evaluation
    avg_list=[]
    
    #taking name to add to dataframe
    new_user = input('Enter your name: ')
    clear_output()
    
    #adding new user to data frame if it isnt already
    if(new_user not in list(df.index.values)):
        #formatting i needed to do
        add_list=[99.9 for x in range(1,102)]
        temp_df=pd.DataFrame(add_list)
        temp_df=temp_df.transpose()
        temp_df[0]=new_user
        temp_df.rename(columns={0:'user_id'}, inplace=True)
        temp_df=temp_df.set_index('user_id')
        
        #adding new user to dataframe
        df=df.append(temp_df,ignore_index=False)
    
    #getting list of jokes rated by the user to determine if they have rated any jokes yet, new users will be 
    #given the top jokes to rate to get a feel for their preferences
    jokes_rated=[joke for joke,rating in df.loc[new_user].iteritems() if not math.isnan(rating)]
    cont='yes'
    
    if(int(len(jokes_rated))==0):
        for text,joke_num in funniest():
            cont=input('Still want to hear a joke (yes to continue, no to stop):')
            clear_output()
            if(cont=='yes'):
                
                #printing joke
                print(text)
                
                #taking input
                new_rating=input('what rating would you give this joke (-10.0 to 10.0)')
                
                #placing rating data into dataframe
                if(-10.0<=int(new_rating)<=10.0):
                    df.loc[new_user,joke_num]=int(new_rating)
                else:
                    print('invalid input')
                
                clear_output()
            else:
                break
                
    #continuation process for jokes
    while(cont=='yes'):
        #print(df.loc[new_user])
        avg=df.loc[new_user].mean(axis=0)
        avg_list.append(avg)
        print('avg',avg)
        cont=input('Still want to hear a joke (yes to continue, no to stop):')
        clear_output()
        if(cont=='yes'):
            
            #getting new joke through user based collaborative filtering
            text,joke_num=new_joke_item_filtering(df,new_user)
            
            #printing joke
            print(text)
            
            #taking input
            new_rating=input('what rating would you give this joke (-10.0 to 10.0)')
            
            #placing rating data into dataframe
            if(-10.0<=int(new_rating)<=10.0):
                df.loc[new_user,joke_num]=int(new_rating)
            else:
                print('invalid input')
            clear_output()
    
    #performance evalutation through slope   
    if(len(avg_list)>=2):
        z=np.polyfit(np.arange(0,len(avg_list)),avg_list,1)
        print("slope of average over time in this instance", z[0])
    
    #need to return the data frame to keep it updated (will reset every time this notebook is restarted)    
    return df
    
#execution of main function
df=item_based_filtering(df)

slope of average over time in this instance -0.20548245614


Above we have a recommendation system for jokes based on item based collaborative filtering. Similar jokes are calcuated after every rating of the joke to ensure the most accurate suggestions. There is no lag between either recommendation systems so both work very well.